# Streamlit LLM Chatbot — Multi‑provider
This app provides a Streamlit UI for chatting with various LLM providers (OpenAI, Google Gemini, Anthropic, Groq, Hugging Face Inference API). Set your API keys as environment variables or Streamlit secrets.


# Installation (commented)
# - Run in terminal if needed
# - pip install streamlit openai anthropic google-generativeai groq huggingface_hub

In [ ]:
# !pip install streamlit openai anthropic google-generativeai groq huggingface_hub tiktoken


# App: imports and env

In [ ]:
import os
import json
import time
import streamlit as st


# Helpers: provider availability checks

In [ ]:
def has_key(name: str) -> bool:
    # Prefer Streamlit secrets, then env var
    try:
        return bool(st.secrets.get(name) or os.environ.get(name))
    except Exception:
        return bool(os.environ.get(name))


# Helpers: client factories (lazy imports)

In [ ]:
def get_openai_client():
    try:
        from openai import OpenAI
        api_key = st.secrets.get('OPENAI_API_KEY', None) if hasattr(st, 'secrets') else None
        if not api_key:
            api_key = os.environ.get('OPENAI_API_KEY')
        if api_key:
            os.environ['OPENAI_API_KEY'] = api_key
        return OpenAI()
    except Exception as e:
        raise RuntimeError(f"OpenAI client error: {e}")

def get_gemini_model(model_name='gemini-1.5-flash'):
    import google.generativeai as genai
    api_key = st.secrets.get('GEMINI_API_KEY', None) if hasattr(st, 'secrets') else None
    if not api_key:
        api_key = os.environ.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
    return genai.GenerativeModel(model_name)

def get_anthropic_client():
    import anthropic
    api_key = st.secrets.get('ANTHROPIC_API_KEY', None) if hasattr(st, 'secrets') else None
    if not api_key:
        api_key = os.environ.get('ANTHROPIC_API_KEY')
    if api_key:
        os.environ['ANTHROPIC_API_KEY'] = api_key
    return anthropic.Anthropic()

def get_groq_client():
    from groq import Groq
    api_key = st.secrets.get('GROQ_API_KEY', None) if hasattr(st, 'secrets') else None
    if not api_key:
        api_key = os.environ.get('GROQ_API_KEY')
    return Groq(api_key=api_key)

def get_hf_client():
    from huggingface_hub import InferenceClient
    token = st.secrets.get('HUGGINGFACEHUB_API_TOKEN', None) if hasattr(st, 'secrets') else None
    if not token:
        token = os.environ.get('HUGGINGFACEHUB_API_TOKEN')
    return InferenceClient(token=token)


# UI — Sidebar configuration

In [ ]:
st.set_page_config(page_title="LLM Chatbot", page_icon="💬", layout="centered")
st.title("💬 LLM Chatbot — Multi‑provider")

with st.sidebar:
    st.header("Provider & Model")
    provider = st.selectbox(
        "Choose provider",
        [
            "OpenAI",
            "Gemini",
            "Anthropic",
            "Groq",
            "HuggingFace",
        ],
        index=0,
    )
    temperature = st.slider("temperature", 0.0, 1.5, 0.7, 0.1)
    max_tokens = st.slider("max tokens", 64, 2048, 256, 32)
    st.caption("Set API keys in st.secrets or environment variables.")

    if provider == "OpenAI":
        model = st.text_input("Model", value="gpt-4o-mini")
        key_ok = has_key('OPENAI_API_KEY')
    elif provider == "Gemini":
        model = st.text_input("Model", value="gemini-1.5-flash")
        key_ok = has_key('GEMINI_API_KEY')
    elif provider == "Anthropic":
        model = st.text_input("Model", value="claude-3-5-sonnet-20241022")
        key_ok = has_key('ANTHROPIC_API_KEY')
    elif provider == "Groq":
        model = st.text_input("Model", value="llama-3.1-8b-instant")
        key_ok = has_key('GROQ_API_KEY')
    else:
        model = st.text_input("Model", value="Qwen/Qwen2.5-1.5B-Instruct")
        key_ok = has_key('HUGGINGFACEHUB_API_TOKEN')

    if not key_ok:
        st.warning("Missing API key for selected provider. Set it in secrets or env.")

system_prompt = st.text_area("System instruction (optional)", value="You are a helpful assistant.")


# Initialize chat history

In [ ]:
if 'messages' not in st.session_state:
    st.session_state.messages = []  # list of {role, content}


# Chat input and display

In [ ]:
for m in st.session_state.messages:
    with st.chat_message(m['role']):
        st.markdown(m['content'])

user_msg = st.chat_input("Type your message…")


# Dispatch call to chosen provider

In [ ]:
def call_provider(provider: str, model: str, messages, temperature: float, max_tokens: int) -> str:
    # messages is list of dicts [{'role': 'system'|'user'|'assistant', 'content': '...'}]
    if provider == 'OpenAI':
        client = get_openai_client()
        resp = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return resp.choices[0].message.content
    if provider == 'Gemini':
        model_obj = get_gemini_model(model)
        # Gemini expects flattened text history; we'll join for simplicity
        prompt = "\n\n".join([f"{m['role']}: {m['content']}" for m in messages])
        out = model_obj.generate_content(prompt)
        return getattr(out, 'text', str(out))
    if provider == 'Anthropic':
        client = get_anthropic_client()
        # Convert messages to Anthropic format (system separate)
        sys = None
        conv = []
        for m in messages:
            if m['role'] == 'system':
                sys = m['content']
            elif m['role'] in ('user', 'assistant'):
                conv.append({'role': m['role'], 'content': m['content']})
        resp = client.messages.create(
            model=model,
            max_tokens=max_tokens,
            temperature=temperature,
            system=sys,
            messages=conv,
        )
        return resp.content[0].text if getattr(resp, 'content', None) else str(resp)
    if provider == 'Groq':
        client = get_groq_client()
        resp = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return resp.choices[0].message.content
    if provider == 'HuggingFace':
        hf = get_hf_client()
        # Use chat_completion if supported, else fall back to text_generation
        try:
            resp = hf.chat_completion(model=model, messages=messages, max_tokens=max_tokens, temperature=temperature)
            return resp.choices[0].message['content'] if hasattr(resp.choices[0], 'message') else resp.choices[0]['message']['content']
        except Exception:
            # Flatten messages for text_generation
            prompt = "\n".join([f"{m['role']}: {m['content']}" for m in messages])
            out = hf.text_generation(model=model, inputs=prompt, max_new_tokens=max_tokens, temperature=temperature)
            return out
    raise ValueError(f"Unsupported provider: {provider}")


# Handle user input

In [ ]:
if user_msg:
    # Build message list for provider
    msgs = []
    if system_prompt:
        msgs.append({'role': 'system', 'content': system_prompt})
    msgs.extend(st.session_state.messages)
    msgs.append({'role': 'user', 'content': user_msg})

    with st.chat_message('user'):
        st.markdown(user_msg)
    with st.chat_message('assistant'):
        placeholder = st.empty()
        try:
            reply = call_provider(provider, model, msgs, temperature, max_tokens)
        except Exception as e:
            reply = f"Error: {e}"
        placeholder.markdown(reply)
    st.session_state.messages.append({'role': 'user', 'content': user_msg})
    st.session_state.messages.append({'role': 'assistant', 'content': reply})


# Notes
# - To run: streamlit run streamlit-chatbot.ipynb (requires Streamlit >= 1.38 support for notebooks) or export to .py via jupytext/nbconvert.
# - Set API keys: OPENAI_API_KEY, GEMINI_API_KEY, ANTHROPIC_API_KEY, GROQ_API_KEY, HUGGINGFACEHUB_API_TOKEN.